In [1]:
import requests, pandas as pd, re
from bs4 import BeautifulSoup

# census api
base = "https://api.census.gov/data/timeseries/intltrade/imports/hs"
params = {
    "get": "I_COMMODITY,I_COMMODITY_SDESC,GEN_VAL_YR",
    "COMM_LVL": "HS10",
    "CTY_CODE": "3510",        # Brazil
    "time": "2024-12"         # only December’s year‑to‑date figure
}
r = requests.get(base, params=params, timeout=60)
r.raise_for_status()

imports = pd.DataFrame(r.json()[1:], columns=r.json()[0])
imports["value"] = pd.to_numeric(imports["GEN_VAL_YR"])
imports["hs8"]   = imports["I_COMMODITY"].str.slice(0, 8)  # trim to HS‑8
total_us_imports = imports["value"].sum()
imports_hs8 = imports.groupby('hs8').agg({
    'I_COMMODITY_SDESC': 'first',
    'value': 'sum'
    }).reset_index()


# exclude re-imports
imports_hs8 = imports_hs8[ imports_hs8.hs8 != '98010010' ] 

# executive order
eo_url = "https://www.whitehouse.gov/presidential-actions/2025/07/addressing-threats-to-the-us/"
html = requests.get(eo_url, timeout=60).text
soup = BeautifulSoup(html, "lxml")

code_pattern = re.compile(r"\d{4}\.\d{2}\.\d{2}")
codes = set(m.group() for m in code_pattern.finditer(soup.get_text()))
annex = pd.DataFrame({"hs8": [c.replace(".", "") for c in codes]})   # strip dots

# merge
covered   = imports_hs8.merge(annex, on="hs8", how="inner")
covered_val = covered["value"].sum()
share_exempt = covered_val / total_us_imports

In [2]:
print(f"US imports from Brazil, 2024 (Census basis): ${total_us_imports:,.0f}")
print(f"Value explicitly *excluded* in Annex I:         ${covered_val:,.0f}")
print(f"Value *not excluded* in Annex I:         ${total_us_imports - covered_val:,.0f}")
print(f"Share excluded:                                {share_exempt:.2%}")
print(f"Share subject to 40 % surcharge:               {(1-share_exempt):.2%}")

US imports from Brazil, 2024 (Census basis): $42,348,411,089
Value explicitly *excluded* in Annex I:         $18,400,222,546
Value *not excluded* in Annex I:         $23,948,188,543
Share excluded:                                43.45%
Share subject to 40 % surcharge:               56.55%


In [3]:
# Top 15 exempt lines, to eyeball the reconciliation
top = covered.sort_values('value', ascending=False).head(15)
print(top)

          hs8                                  I_COMMODITY_SDESC       value
11   27090020  CRUDE PETROLEUM TEST 25 DEGREES API OR MORE NESOI  4970263973
79   72011000     NONALLOY PIG IRON 0.5 PRCNT OR LESS PHOSPHORUS  1536412085
10   27090010       CRUDE PETROLEUM TESTING UNDER 25 DEGREES API  1510827641
65   47032900  CHEM WDPULP SULFATE EX DSSLVNG GR NONCONF SEMI...  1338868639
341  88023001   NEW TURBOFAN PLANES,NON-MILITARY,(4536-15000 KG)  1069140367
12   27101215        MOTOR FUEL, NOT CONTAINING BIODIESEL, NESOI   787265187
16   27101906  HEAVY FL OIL,DISTL/RESD,LT 25 DEG,NT BIODSEL N...   785458518
342  88024001  NEW PASSENGR TRANSPORT,NON-MILITARY,WT GT 15,0...   752240710
3    20091225  ORANGE JUICE UNFERMNTD NOT FROZ NOT CONCENTRTD...   663210117
7    26011200    IRON ORE AND CONCENTRATES, AGGLOMERATED PELLETS   420952147
34   28182000  ALUMINUM OXIDE, EXCEPT ARTIFICIAL CORUNDUM, NESOI   371670695
72   68029900           OTHER STONE MONUMENTS, BASES AND MARKERS   369665559

In [4]:
# Top 15 non-exempt lines
noncovered_set = set(imports_hs8.hs8).difference(set(covered.hs8))
noncovered = imports_hs8[ [x in noncovered_set for x in imports_hs8.hs8] ]
top_noncovered = noncovered.sort_values('value', ascending=False).head(15)
print(top_noncovered)

           hs8                                  I_COMMODITY_SDESC       value
4254  98010010  GOODS OF CHAPTER 28 RETURNED AFTER BEING EXPORTED  2403779860
150   09011100   COFFEE, ORGANIC, ARABICA, NOT DECAF, NOT ROASTED  1873630841
2520  72071200  SEMIFIN IR/STL LT 0.25%CARB CROS-SECT WD GT=4X...  1676283481
2589  72249000  SMFD AL STL NT SS OR TS,REC/SQ,WIDTH 4X THKNS AOV   509867463
3149  84295110  SHOV LDR,REAR ENG MT,4WH DR,CAP1.5M3UNDER2.2M3...   470563671
8     02023080           MEAT OF BOVINES, BONELESS, FROZEN, NESOI   462359371
272   17011450  CANE SUGAR,RAW, NO ADDED FLVORING/COLORING, NESOI   439592941
7     02023050   MEAT,BVINE BONELESS FRZN US NTE 3 EXC PRCSD, RIB   433991471
1354  44091040            PINE STANDARD WOOD MOLDING, END-JOINTED   350512417
262   16025021  MT BOV ANML NESOI NO VEG PREP AIRTITE CNTR LT 1KG   346999676
232   15021000                                   TALLOW, INEDIBLE   345881394
3146  84292000               GRADERS AND LEVELERS, SELF-PROPELLE